# Prediction of the "classification" label 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, RobertaModel, RobertaTokenizer, ViTModel, BlipProcessor, BlipForQuestionAnswering , CLIPProcessor, CLIPModel, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoModelForImageClassification, AutoImageProcessor
from transformers import BeitImageProcessor, BeitForImageClassification
import pickle 
import torch.nn as nn
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
df_combined = pd.read_csv('df_combined.csv')
df_combined

In [ ]:
def get_model_save_location(model_path):
    parts = model_path.split('/', 1)  # Split at the first "/" encountered
    return parts[1] if len(parts) > 1 else model_path

def get_multimodal_model_save_location(nlp_model_path, image_model_path, operation):
    nlp_parts = nlp_model_path.split('/', 1)
    nlp_tmp = nlp_parts[1] if len(nlp_parts) > 1 else nlp_model_path
    image_parts = image_model_path.split('/', 1)
    image_tmp = image_parts[1] if len(image_parts) > 1 else image_model_path
    return f"{nlp_tmp}_{image_tmp}_{operation}"

def save_model_path(model_name):
    return f"./trained_models/{model_name}.pt"

def save_predictions_path(model_name):
    return f"./trained_results/{model_name}.csv"

## NLP prediction

https://huggingface.co/BAAI/bge-reranker-large

## Version 3

In [ ]:
# Create a PyTorch dataset
class ClassifierDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts.to_numpy()
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len        

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create data loaders
def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    ds = ClassifierDataset(
        texts=texts,
        labels=labels,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size, num_workers=4)

class NLPClassifier(nn.Module):
    def __init__(self, n_classes, pretrained_model_base):
        super(NLPClassifier, self).__init__()
        self.pretrained = pretrained_model_base
        self.drop = nn.Dropout(p=0.3)
        self.hidden = nn.Linear(self.pretrained.config.hidden_size, 128)  # Change 128 to your desired hidden layer size
        self.out = nn.Linear(128, n_classes)

    def forward(self, input_ids, attention_mask):
        text_output = self.pretrained(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        text_pooled_output = text_output.last_hidden_state[:, 0, :]
        
        output = self.drop(text_pooled_output)
        output = nn.ReLU()(self.hidden(output))
        # return self.out(output)
        return torch.nn.functional.log_softmax(self.out(output), dim=1)

In [ ]:
# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model_path = 'roberta-base'
# model_path = 'BAAI/bge-reranker-large'
# model_path = 'openbmb/Eurus-RM-7b' - has problems, TODO fix
# model_path = 'facebook/bart-large-cnn'

tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModel.from_pretrained(model_path).to(device)

data_subset = df_combined[:50]

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(data_subset['classification_by_editorial'])

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data_subset['text'], encoded_labels, test_size=0.2, random_state=42)

BATCH_SIZE = 10  # maximum for BGE is ~116
MAX_LEN = 256

train_data_loader = create_data_loader(train_texts, train_labels, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_texts, test_labels, tokenizer, MAX_LEN, BATCH_SIZE)

# Initialize the classifier and optimizer
model = NLPClassifier(len(le.classes_), base_model).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss

# Define the loss function
loss_fn = CrossEntropyLoss().to(device)

# Define the number of training epochs
EPOCHS = 5

train_losses = []
test_losses = []

total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

for epoch in range(EPOCHS):
    print(f'STARTING Epoch {epoch + 1}/{EPOCHS}')
    model.train()
    total_loss = 0

    for batch in tqdm(train_data_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        model.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_data_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_loss = 0

    for batch in tqdm(test_data_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask)

        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

    avg_test_loss = total_loss / len(test_data_loader)
    test_losses.append(avg_test_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {avg_train_loss}, Test Loss: {avg_test_loss}")

# Plotting the training and testing losses
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Testing loss')
plt.legend()
plt.show()

# Evaluation
model.eval()
predictions = []
true_labels = []

for batch in tqdm(test_data_loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    _, preds = torch.max(outputs, 1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Save model:
torch.save(model.state_dict(), save_model_path(get_model_save_location(model_path)))

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(le.inverse_transform(true_labels), le.inverse_transform(predictions)))

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

2 epochs with Roberta model:

```
Accuracy: 0.9721559074299635
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      3908
           1       0.99      0.98      0.98     17162
           2       0.93      0.87      0.90      2410
           3       0.99      0.98      0.98     17570

    accuracy                           0.97     41050
   macro avg       0.95      0.95      0.95     41050
weighted avg       0.97      0.97      0.97     41050
```

5 epochs with BGE-Reranker large

```
Accuracy: 0.9645155000974849
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3870
           1       0.98      0.98      0.98     17201
           2       0.82      0.86      0.84      2374
           3       0.98      0.98      0.98     17587

    accuracy                           0.96     41032
   macro avg       0.92      0.93      0.93     41032
weighted avg       0.97      0.96      0.96     41032
```

5 epochs with BART-large-CNN
```
Accuracy: 0.9893985182296744
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      3870
           1       0.99      0.99      0.99     17201
           2       0.97      0.95      0.96      2374
           3       0.99      0.99      0.99     17587

    accuracy                           0.99     41032
   macro avg       0.98      0.98      0.98     41032
weighted avg       0.99      0.99      0.99     41032
```

### Loading and evaluating trained models

In [ ]:
model = NLPClassifier(len(le.classes_), base_model).to(device)
model.load_state_dict(torch.load(save_model_path(get_model_save_location(model_path))))
model.eval()

In [ ]:
# Evaluation
model.eval()
predictions = []
true_labels = []

for batch in tqdm(test_data_loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    _, preds = torch.max(outputs, 1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(le.inverse_transform(true_labels), le.inverse_transform(predictions)))

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Multimodality

In [ ]:
# Create a PyTorch dataset
class ClassifierDataset(Dataset):
    def __init__(self, texts, labels, images, tokenizer, processor, max_len):
        self.texts = texts.to_numpy()
        self.labels = labels
        self.images = images.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.processor = processor

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        image = Image.open(
            './images/' + str(self.images[idx]) + '.jpg').convert('RGB')
        # image = self.transform(image)
        image = self.processor(image, return_tensors='pt').pixel_values.squeeze(0)

        # print("SHAPE of image: " + str(image.shape))

        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'image': image,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create data loaders
def create_data_loader(texts, labels, tokenizer, images, processor, max_len, batch_size):
    ds = ClassifierDataset(
        texts=texts,
        labels=labels,
        tokenizer=tokenizer,
        images=images,
        processor=processor,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size, num_workers=4)


class MMClassifier(nn.Module):
    # TODO: add hidden layer size as parameter
    def __init__(self, n_classes, nlp_model, image_model, fusion_mode):
        super(MMClassifier, self).__init__()
        self.fusion_mode = fusion_mode

        self.nlp_model = nlp_model
        self.image_model = image_model

        self.hidden_layer_size = 384
        self.hidden_size = self.get_hidden_size()

        self.image_linear = nn.Linear(self.image_model.config.hidden_size, out_features=self.hidden_layer_size)
        self.nlp_linear = nn.Linear(self.nlp_model.config.hidden_size, out_features=self.hidden_layer_size)
        
        self.hidden = nn.Linear(self.hidden_size, 128)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(128, n_classes)

    # Wrapper for the match statement for hidden size
    def get_hidden_size(self):
        match self.fusion_mode:
            case 'concat':
                return self.hidden_layer_size * 2
            case 'mul' | 'add' | 'einsum':
                return self.hidden_layer_size
            case _:
                raise ValueError("Invalid fusion mode")

    def get_fused_output(self, text_pooled_output, image_pooled_output):
        match self.fusion_mode:
            case 'concat':
                return torch.cat((text_pooled_output, image_pooled_output), dim=1)
            case 'mul':
                return text_pooled_output.mul(image_pooled_output)
            case 'add':
                return (text_pooled_output + image_pooled_output)/2
            # TODO: using einsum  (need to work out some bugs) 
            case 'einsum':
                return torch.einsum('ij,ij->ij', text_pooled_output, image_pooled_output)
            case _:
                raise ValueError("Invalid fusion mode")

    def forward(self, input_ids, attention_mask, image):
        text_output = self.nlp_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        image_output = self.image_model(image)

        text_pooled_output = text_output.last_hidden_state[:, 0, :]
        image_pooled_output = image_output.hidden_states[-1][:, 0, :]
        
        text_output = nn.ReLU()(self.nlp_linear(text_pooled_output))
        image_output = nn.ReLU()(self.image_linear(image_pooled_output))

        combined = self.get_fused_output(text_output, image_output)

        # print("SHAPE of combined: " + str(combined.shape))

        output = self.drop(combined)
        output = nn.ReLU()(self.hidden(output))
        # return self.out(output)
        return torch.nn.functional.log_softmax(self.out(output), dim=1)

In [ ]:
# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

nlp_model_path = 'roberta-base' # 'BAAI/bge-reranker-large' # 'facebook/bart-large-cnn' # 'roberta-base'
image_model_path = 'google/vit-base-patch16-224' # 'microsoft/resnet-50' # 'google/vit-base-patch16-224'
fusion_mode = 'mul'

# Load pretrained model/tokenizer
tokenizer = AutoTokenizer.from_pretrained(nlp_model_path)
nlp_model = AutoModel.from_pretrained(nlp_model_path).to(device)
image_model = AutoModel.from_pretrained(image_model_path).to(device)
processor = AutoImageProcessor.from_pretrained(image_model_path)
image_model.config.output_hidden_states = True

data_subset = df_combined#[:10]

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(data_subset['classification_by_editorial'])

# Split data into training and test sets
train_texts, test_texts, train_images, test_images, train_labels, test_labels = train_test_split(
    data_subset['text'], data_subset["id"], encoded_labels, test_size=0.2, random_state=42)

BATCH_SIZE = 200  # 50 consumes 22GB of VRAM
MAX_LEN = 256

train_data_loader = create_data_loader(train_texts, train_labels, tokenizer, train_images, processor, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_texts, test_labels, tokenizer, test_images, processor, MAX_LEN, BATCH_SIZE)

# Initialize the classifier and optimizer
model = MMClassifier(len(le.classes_), nlp_model, image_model, fusion_mode).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
# Define the loss function
loss_fn = CrossEntropyLoss().to(device)

# Define the number of training epochs
EPOCHS = 5

train_losses = []
test_losses = []

total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

for epoch in range(EPOCHS):
    print(f'STARTING Epoch {epoch + 1}/{EPOCHS}')
    model.train()
    total_loss = 0

    for batch in tqdm(train_data_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        images = batch["image"].to(device)

        model.zero_grad()
        outputs = model(input_ids, attention_mask, images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_data_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_loss = 0

    for batch in tqdm(test_data_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        images = batch["image"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask, images)

        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

    avg_test_loss = total_loss / len(test_data_loader)
    test_losses.append(avg_test_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {avg_train_loss}, Test Loss: {avg_test_loss}")

# Plotting the training and testing losses
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Testing loss')
plt.legend()
plt.show()

# Evaluation
model.eval()
predictions = []
true_labels = []

for batch in tqdm(test_data_loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    images = batch["image"].to(device)
    
    with torch.no_grad():
            outputs = model(input_ids, attention_mask, images)

    _, preds = torch.max(outputs, 1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())
    
# Save model:
torch.save(model.state_dict(), save_model_path(get_multimodal_model_save_location(nlp_model_path, image_model_path, fusion_mode)))

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(le.inverse_transform(true_labels), le.inverse_transform(predictions)))

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

5 epochs of concatenation of the Roberta model and the ViT model:

```
Accuracy: 0.986181516864886
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.95      0.98      0.96      3870
 left_wing_outside_US       0.99      0.99      0.99     17201
     right_wing_in_US       0.97      0.93      0.95      2374
right_wing_outside_US       0.99      0.99      0.99     17587

             accuracy                           0.99     41032
            macro avg       0.98      0.97      0.97     41032
         weighted avg       0.99      0.99      0.99     41032
```

5 epochs concat Bart-Large-CNN + ViT:
```
Accuracy: 0.9896419780166216
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.97      0.98      0.97      3958
 left_wing_outside_US       0.99      0.99      0.99     17133
     right_wing_in_US       0.96      0.96      0.96      2441
right_wing_outside_US       0.99      0.99      0.99     17499

             accuracy                           0.99     41031
            macro avg       0.98      0.98      0.98     41031
         weighted avg       0.99      0.99      0.99     41031
```

5 epochs concat BGE-Reranker + ViT:
```
Accuracy: 0.9739709000511808
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.93      0.95      0.94      3958
 left_wing_outside_US       0.98      0.98      0.98     17133
     right_wing_in_US       0.91      0.89      0.90      2441
right_wing_outside_US       0.98      0.98      0.98     17499

             accuracy                           0.97     41031
            macro avg       0.95      0.95      0.95     41031
         weighted avg       0.97      0.97      0.97     41031
```

5 epochs plus Bart-Large-CNN + ViT
```
Accuracy: 0.9897394652823475
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.98      0.98      0.98      3958
 left_wing_outside_US       0.99      0.99      0.99     17133
     right_wing_in_US       0.96      0.96      0.96      2441
right_wing_outside_US       0.99      0.99      0.99     17499

             accuracy                           0.99     41031
            macro avg       0.98      0.98      0.98     41031
         weighted avg       0.99      0.99      0.99     41031
```

5 epochs mul Bart-Large-CNN + ViT
```
Accuracy: 0.9898125807316419
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.97      0.98      0.98      3958
 left_wing_outside_US       0.99      0.99      0.99     17133
     right_wing_in_US       0.97      0.96      0.96      2441
right_wing_outside_US       0.99      0.99      0.99     17499

             accuracy                           0.99     41031
            macro avg       0.98      0.98      0.98     41031
         weighted avg       0.99      0.99      0.99     41031
```

5 epochs mul  Reranker + ViT
```
Accuracy: 0.952645560673637
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.90      0.88      0.89      3958
 left_wing_outside_US       0.97      0.97      0.97     17133
     right_wing_in_US       0.79      0.80      0.80      2441
right_wing_outside_US       0.97      0.97      0.97     17499

             accuracy                           0.95     41031
            macro avg       0.91      0.91      0.91     41031
         weighted avg       0.95      0.95      0.95     41031
```

5 epochs add Reranker + ViT
```
Accuracy: 0.9728497964953328
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.93      0.94      0.94      3958
 left_wing_outside_US       0.98      0.98      0.98     17133
     right_wing_in_US       0.90      0.88      0.89      2441
right_wing_outside_US       0.98      0.99      0.98     17499

             accuracy                           0.97     41031
            macro avg       0.95      0.95      0.95     41031
         weighted avg       0.97      0.97      0.97     41031
```

5 epochs add Roberta + ViT
```
Accuracy: 0.9869367063927275
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.96      0.98      0.97      3958
 left_wing_outside_US       0.99      0.99      0.99     17133
     right_wing_in_US       0.96      0.94      0.95      2441
right_wing_outside_US       0.99      0.99      0.99     17499

             accuracy                           0.99     41031
            macro avg       0.98      0.97      0.98     41031
         weighted avg       0.99      0.99      0.99     41031
```

5 epochs mul Roberta + ViT
```
Accuracy: 0.9834027930101631
Classification Report:
                       precision    recall  f1-score   support

      left_wing_in_US       0.95      0.97      0.96      3958
 left_wing_outside_US       0.99      0.99      0.99     17133
     right_wing_in_US       0.96      0.92      0.94      2441
right_wing_outside_US       0.99      0.99      0.99     17499

             accuracy                           0.98     41031
            macro avg       0.97      0.97      0.97     41031
         weighted avg       0.98      0.98      0.98     41031
```

In [ ]:
model = MMClassifier(len(le.classes_), nlp_model, image_model, fusion_mode).to(device)
model.load_state_dict(torch.load(save_model_path(get_multimodal_model_save_location(nlp_model_path, image_model_path, fusion_mode))))
model.eval()

In [ ]:
# Evaluation
model.eval()
predictions = []
true_labels = []

for batch in tqdm(test_data_loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    images = batch["image"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask, images)

    _, preds = torch.max(outputs, 1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(le.inverse_transform(true_labels), le.inverse_transform(predictions)))

# Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

### Pretrained multimodal models:

# Testing fusion approaches

In [ ]:
# create two tensort of the size 768 and 768
a = torch.randn(6, 768)
b = torch.randn(6, 768)

In [ ]:
# Basic element-wise multiplication

d = a.mul(b)
d.shape

In [ ]:
e = torch.matmul(a, b.reshape(768, 6))
e

In [ ]:
torch.matmul(a, b.T)

In [ ]:
(a * b) == d

In [ ]:
(a + b).shape

In [ ]:
f = torch.einsum('ik,jk->ij', a, b)
f

In [ ]:
f == (a * b)

In [ ]:
# compute einsum and normalise it
g = torch.einsum('ij,jk->ik', a, b.T)
g = torch.nn.functional.normalize(g, p=2, dim=1)
g

In [ ]:
i = torch.einsum('ij,jk->ik', a, b.reshape(768, 6))
i = torch.nn.functional.normalize(g, p=2, dim=1)
i

In [ ]:
h = torch.ger(a[0], b[0])
h.shape

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = AutoModel.from_pretrained('google/vit-base-patch16-224')
model.config.output_hidden_states = True

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [ ]:
inputs

In [ ]:
logits.shape

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
outputs.last_hidden_state.squeeze(0).shape

In [ ]:
outputs.last_hidden_state.squeeze(0)[0, :].shape

In [ ]:
tmp = processor(images=image, return_tensors="pt").pixel_values
### reshape tmp to 3, 224, 224
# mp = tmp.squeeze(0)
tmp.shape

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from PIL import Image
import requests
import torch

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = AutoModel.from_pretrained("microsoft/resnet-50")
model.config.output_hidden_states = True

inputs = processor(image, return_tensors="pt")

# TODO: add pooling

In [ ]:
model.config

In [ ]:
states = model(**inputs).last_hidden_state
states.shape

In [ ]:
states[0, :, :].shape

In [ ]:
from transformers import MobileViTFeatureExtractor, MobileViTForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoImageProcessor.from_pretrained("apple/mobilevit-small")
model = AutoModel.from_pretrained("apple/mobilevit-small")

inputs = feature_extractor(images=image, return_tensors="pt")

outputs = model(**inputs)
# TODO - add pooling

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
model.config

In [ ]:
from transformers import BeitImageProcessor, BeitForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = BeitImageProcessor.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')
model.config.output_hidden_states = True

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
outputs

In [ ]:
outputs.hidden_states[-1].shape

In [ ]:
inputs.pixel_values.shape

In [ ]:
model.config.hidden_size